In [3]:
import torch
from torch.nn import Softplus
from pina import PINN, Plotter, LabelTensor
from pina.model import FeedForward
from problems.burgers_with_data import Burgers1D
import matplotlib.pyplot as plt

# Feature definition
class myFeature(torch.nn.Module):
    """
    Feature: sin(pi*x)
    """
    def __init__(self, idx):
        super(myFeature, self).__init__()
        self.idx = idx

    def forward(self, x):
        return LabelTensor(torch.sin(torch.pi * x.extract(['x'])), ['sin(x)'])

# Manually set arguments
id_run = 0  # Example value
features = 0  # Example value (0 for no extra features, 1 for extra features)
save = True  # Set to True to save, False to load

feat = [myFeature(0)] if features else []

ntotal = 2000
cut_Data = 2

burgers_problem = Burgers1D(ntotal, cut_Data)
model = FeedForward(
    layers=[30, 20, 10, 5],
    output_variables=burgers_problem.output_variables,
    input_variables=burgers_problem.input_variables,
    func=Softplus,
    extra_features=feat,
)

pinn = PINN(
    burgers_problem,
    model,
    lr=0.006,
    error_norm='mse',
    regularizer=0
)

if save:
    pinn.span_pts(
        {'n': 100, 'mode': 'grid', 'variables': 't'},
        {'n': 20, 'mode': 'grid', 'variables': 'x'},
        locations=['D', 'gamma1', 'gamma2', 't0']
    )
    pinn.train(6000, 1)
    pinn.save_state(f'pina.burger_dis1.{id_run}.{features}')
else:
    pinn.load_state(f'pina.burger_dis1.{id_run}.{features}')
    plotter = Plotter()
    plotter.plot_same_training_test_data(pinn)
    plotter.plot_loss(pinn)

# Example of how to display plots in Jupyter
plt.show()


TypeError: nil_dirichlet() missing 1 required positional argument: 'output_'

In [3]:
import torch
from torch.nn import Softplus

from pina import PINN, Plotter, LabelTensor
from pina.model import FeedForward
# from problems.burgers import Burgers1D
from examples.problems.my_burger import Burgers1D


class myFeature(torch.nn.Module):
    """
    Feature: sin(pi*x)
    """
    def __init__(self, idx):
        super(myFeature, self).__init__()
        self.idx = idx

    def forward(self, x):
        return LabelTensor(torch.sin(torch.pi * x.extract(['x'])), ['sin(x)'])
    


save = True
id_run = 0
features =  0

feat = [myFeature(0)] if features else []
# ntotal = 2000
# cut_Data = 2

burgers_problem = Burgers1D()
model = FeedForward(
    layers=[30, 20, 10, 5],
    output_variables=burgers_problem.output_variables,
    input_variables=burgers_problem.input_variables,
    func=Softplus,
    extra_features=feat,
)
pinn = PINN(
    burgers_problem,
    model,
    lr=0.006,
    error_norm='mse',
    regularizer=0)
if save:
    pinn.span_pts(
        {'n': 100, 'mode': 'grid', 'variables': 't'},
        {'n': 20, 'mode': 'grid', 'variables': 'x'},
        locations=['D','gamma1', 'gamma2', 't0'])
    pinn.train(6000, 100)
    pinn.save_state('pina.burger_dis1.{}.{}'.format(id_run, features))
else:
    pinn.load_state('pina.burger_dis1.{}.{}'.format(id_run, features))
    plotter = Plotter()
    # plotter.plot_same_training_test_data(pinn)
    plotter.plot_loss(pinn)

# plt.show()


              sum          gamma1nil_di gamma2nil_di t0initial_co Dburger_equa 
[epoch 00000] 8.491638e-01 4.201743e-02 4.237496e-02 7.647709e-01 5.395496e-07 
              sum          gamma1nil_di gamma2nil_di t0initial_co Dburger_equa 
[epoch 00001] 7.259781e-01 2.287766e-02 2.288103e-02 6.802188e-01 5.866477e-07 
              sum          gamma1nil_di gamma2nil_di t0initial_co Dburger_equa 
[epoch 00100] 3.495447e-01 4.087321e-02 4.061756e-02 2.675442e-01 5.096614e-04 
              sum          gamma1nil_di gamma2nil_di t0initial_co Dburger_equa 
[epoch 00200] 3.311781e-01 3.264039e-02 3.905104e-02 2.444834e-01 1.500333e-02 
              sum          gamma1nil_di gamma2nil_di t0initial_co Dburger_equa 
[epoch 00300] 1.039853e-01 8.212783e-03 9.715864e-03 5.323435e-02 3.282226e-02 
              sum          gamma1nil_di gamma2nil_di t0initial_co Dburger_equa 
[epoch 00400] 6.303643e-02 5.578548e-03 1.257446e-02 1.664144e-02 2.824198e-02 
              sum          gamma1nil_di 

KeyboardInterrupt: 

In [5]:
import sys
import numpy as np
import torch
from torch.nn import ReLU, Tanh, Softplus

from pina import PINN, LabelTensor, Plotter
from pina.model import FeedForward
from pina.adaptive_functions import AdaptiveSin, AdaptiveCos, AdaptiveTanh
from examples.problems.stokes import Stokes

id_run = 0
save = True

stokes_problem = Stokes()
model = FeedForward(
    layers=[10, 10, 10, 10],
    output_variables=stokes_problem.output_variables,
    input_variables=stokes_problem.input_variables,
    func=Softplus,
)
pinn = PINN(
    stokes_problem,
    model,
    lr=0.006,
    error_norm='mse',
    regularizer=1e-8)
if save:
    pinn.span_pts(200, 'grid', locations=['gamma_top', 'gamma_bot', 'gamma_in', 'gamma_out'])
    pinn.span_pts(2000, 'random', locations=['D'])
    pinn.train(10, 10)
    with open('stokes_history_{}.txt'.format(id_run), 'w') as file_:
        for i, losses in pinn.history_loss.items():
            file_.write('{} {}\n'.format(i, sum(losses)))
    pinn.save_state('pina.stokes')
else:
    pinn.load_state('pina.stokes')
    plotter = Plotter()
    plotter.plot(pinn, components='ux')
    plotter.plot(pinn, components='uy')
    plotter.plot(pinn, components='p')


              sum          gamma_topwal gamma_botwal gamma_outout gamma_ininle Dmomentum    Dcontinuity  
[epoch 00000] 2.569453e+00 3.615445e-03 3.647497e-03 2.771125e-01 2.285069e+00 2.664468e-06 5.603756e-06 
              sum          gamma_topwal gamma_botwal gamma_outout gamma_ininle Dmomentum    Dcontinuity  
[epoch 00001] 2.220447e+00 5.181166e-04 5.175415e-04 1.913473e-01 2.028052e+00 4.195438e-06 7.154890e-06 
[epoch 00010] 1.364429e+00 3.929497e-01 3.830320e-01 4.819550e-02 5.399247e-01 1.376403e-04 1.892310e-04 
